# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [58]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [59]:
X_train = pd.read_csv('X_train.txt', sep=',')
X_test = pd.read_csv('X_test.txt',sep=',')
y_train = pd.read_csv('y_train.txt', header=None)
y_test = pd.read_csv('y_test.txt', header=None, delim_whitespace=True)

In [60]:
# coferindo o tamanho das bases
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7352, 563)
(2947, 563)
(7352, 1)
(2947, 1)


### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [61]:
X_train_final, X_valid, y_train_final, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1729)

In [62]:
y_train_final.shape

(5514, 1)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [70]:
clf = DecisionTreeClassifier(max_depth=4, random_state=1729).fit(X_train_final, y_train_final)

importances = clf.feature_importances_

# nome das variáveis
feature_names = X_train_final.columns if isinstance(X_train_final, pd.DataFrame) else [f'Feature {i}' for i in range(X_train_final.shape[1])]

# colocando em um df e ordenando
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# variável das top 3 selecionadas
top_3_features = importance_df.head(3)

print("As 3 variáveis mais importantes são:\n",top_3_features)


As 3 variáveis mais importantes são:
                              Feature  Importance
560             angle(X,gravityMean)    0.280288
395  fBodyAccJerk-bandsEnergy()-1,24    0.253431
561             angle(Y,gravityMean)    0.172169


In [ ]:
feature_names

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [71]:
#lista das variáveis selecionadas acima
top_features = ['angle(X,gravityMean)', 'fBodyAccJerk-bandsEnergy()-1,24', 'angle(Y,gravityMean)']
X_train_top = X_train_final[top_features]
X_valid_top = X_valid[top_features]

#ccp_alpha
clf = DecisionTreeClassifier(random_state=2360873)
path = clf.cost_complexity_pruning_path(X_train_top, y_train_final)
ccp_alphas = path.ccp_alphas



In [76]:
clf

DecisionTreeClassifier(ccp_alpha=0.1853135001888231, random_state=2360873)

In [77]:
clfs = []
train_scores = []
valid_scores = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X_train_top, y_train_final)
    clfs.append(clf)
    
    train_scores.append(clf.score(X_train_top, y_train_final))
    valid_scores.append(clf.score(X_valid_top, y_valid))


best_index = np.argmax(valid_scores)
best_ccp_alpha = ccp_alphas[best_index]

best_clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=best_ccp_alpha).fit(X_train_top, y_train_final)
valid_accuracy = best_clf.score(X_valid_top, y_valid)

print(f'Melhor valor de ccp_alpha: {best_ccp_alpha}')
print(f'Acurácia na base de validação com o melhor ccp_alpha: {valid_accuracy}')

Melhor valor de ccp_alpha: 0.00018135654697134566
Acurácia na base de validação com o melhor ccp_alpha: 0.8732317736670294


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [65]:
X_test_top = X_test[top_features]

test_accuracy = best_clf.score(X_test_top, y_test)
print(f'Acurácia na base de testes: {test_accuracy}')

Acurácia na base de testes: 0.6844248388191381
